# 3rd NOTEBOOK CI analysis powered by Unet

In [1]:

import os
import sys
sys.path.append(r'.\Modules')

import pandas as pd
import numpy as np
from skimage.io import imshow,imread,imsave 
from PIL import Image
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import segmentation_models as sm
from skimage.filters import gaussian
from skimage.feature import blob_doh
#import openslide
from skimage.transform import resize

from skimage.exposure import match_histograms
import seaborn as sns
import cv2
import slideio

from skimage.exposure import match_histograms

import wholeslideTools
import parallelWholeslideTools
from model_predict import minMaxScale
from model_predict import model_predictions

import multisceneCziTools

def resshow(image,mask,thresh=0.5,number=0):
    """Function to show results with threshold"""
    plt.figure(figsize=(30,10))
    plt.subplot(141)
    imshow(image[number])
    plt.subplot(142)
    imshow(mask[number,:,:,0]>thresh)
    plt.subplot(143)
    imshow(mask[number,:,:,1]>thresh)
    plt.subplot(144)
    imshow(mask[number,:,:,2]>thresh)

Segmentation Models: using `keras` framework.


## Load a table with the name and location of the slide 

In [2]:
working_dir = r"D:\Nick\AFOGdata_P11703_MariaGalardi"
file_names = [f for f in os.listdir(working_dir) if os.path.isfile(os.path.join(working_dir, f))]

# Extract format (the last part after the last full stop) from each file name without extension
formats = [os.path.splitext(file_name)[1][1:] if '.' in file_name else '' for file_name in file_names]

# Create a DataFrame
data = {
    'Folder': [working_dir] * len(file_names),
    'File name': file_names,
    'Format': formats
}
df = pd.DataFrame(data)
# df = df[:1]
df


,Folder,File name,Format
0,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__01.czi,czi
1,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__02.czi,czi
2,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__03.czi,czi
3,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__04.czi,czi
4,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__05.czi,czi
5,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__06.czi,czi
6,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__07.czi,czi
7,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__08.czi,czi
8,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__09.czi,czi
9,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__10.czi,czi


In [3]:
# list of files location to be added in the next line
# df_slide = pd.read_csv("D:/Nick/4thExpAFOG/20220617_TableOfContent.csv",sep=';') #[:1] #[:1] #is to choose how many rows should the program run
df_slide= df.copy()
#df_slide = pd.read_excel("C:/Users/nk20x980/Desktop/test/20220912_ListOfFiles.xlsx")#[:10]
# df_slide["Image path"] = df_slide["Folder"].str.replace("\\", "/") + "/" + df_slide["File name"].astype(str) +"."+ df_slide["Format"]
df_slide["Image path"] = df_slide["Folder"].str.replace("\\", "/") + "/" + df_slide["File name"].astype(str) #+"."+ df_slide["Format"]
df_slide = df_slide.set_index("Image path")
df_slide.head()



C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\464515739.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_slide["Image path"] = df_slide["Folder"].str.replace("\\", "/") + "/" + df_slide["File name"].astype(str) #+"."+ df_slide["Format"]


,Folder,File name,Format
Image path,,,
D:/Nick/AFOGdata_P11703_MariaGalardi/20230928__01.czi,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__01.czi,czi
D:/Nick/AFOGdata_P11703_MariaGalardi/20230928__02.czi,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__02.czi,czi
D:/Nick/AFOGdata_P11703_MariaGalardi/20230928__03.czi,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__03.czi,czi
D:/Nick/AFOGdata_P11703_MariaGalardi/20230928__04.czi,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__04.czi,czi
D:/Nick/AFOGdata_P11703_MariaGalardi/20230928__05.czi,D:\Nick\AFOGdata_P11703_MariaGalardi,20230928__05.czi,czi


In [4]:
#print current directory
print(os.getcwd())

D:\Prateek\MICROSCOPY_AI_Cryoinjuries-main\MICROSCOPY_AI_Cryoinjuries-main


## Load the Unet

In [6]:

path_model = "D:\Prateek\MICROSCOPY_AI_Cryoinjuries-main\MICROSCOPY_AI_Cryoinjuries-main\slide_scanner_15112022.hdf5"


cls_weights = np.array([1,0.25,1.5]) #background, ventricle, injury
jacc_loss = sm.losses.JaccardLoss(class_weights=cls_weights, class_indexes=None, per_image=False, smooth=1e-05)
dice_loss  = sm.losses.DiceLoss()
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (2 * focal_loss)

num_classes = 3
pat=50
lr=0.0001
activation= 'softmax'
loss_func= (total_loss,'dice_loss_plus_2focal_loss')


#model = load_model(path_model, custom_objects={'multi_dice_coef': multi_dice_coef})
model = load_model(path_model,custom_objects={'jaccard_loss':loss_func[0],'dice_loss':dice_loss,'dice_loss_plus_2focal_loss':total_loss})
                            

## Apply Unet to data

In [7]:
make_folders = {"Annotations",
                "Images",
                "Results",
                }
for i in make_folders:
    folder = os.path.join(working_dir, i)
    
    if not os.path.exists(folder):
       os.mkdir(folder)
       print("The folders were created:", i)
    else:
       print("The folder exists already:", folder)

The folders were created: Results
The folders were created: Annotations
The folders were created: Images


In [8]:
for i in df_slide.index:
    try:
        print(i)


        if (i.endswith(".ndpi")) | (i.endswith(".mrxs")):

            slide = parallelWholeslideTools.wholeslideTools(i)
            array = slide.exportAllSections(    cropLoc = 1.3,
                                                downscale = 512,
                                                drop = [],
                                                filterSections = True,
                                                upperStd = 0.25,
                                                lowerStd = 0.04,
                                                arrayOut = True)

        if i.endswith(".czi"):
            # print("test_1")
            slide = multisceneCziTools.multiscene_czi_export(i)
            
            array = minMaxScale(slide.section_toarray())
            
        for im_n,images_i in enumerate(array):
            
            imsave(working_dir+"/Images/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),im_n),images_i)
            # print("test_2")
            
        
        pred = model.predict(array, verbose=1)
        # print("test_3")
        
        for pred_n,pred_i in enumerate(pred):    
            imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)


        df_out = pd.DataFrame([(num,np.count_nonzero(i[:,:,1]>0.5),np.count_nonzero(i[:,:,2]>0.5)) for num,i in enumerate(pred)], columns=["Section Number","Ventricle (Pixel)","CI (Pixel)"])
        print(df_out)
        # print("test_4")
        
        df_out[["Ventricle (µm^2)","CI (µm^2)"]] = df_out[["Ventricle (Pixel)","CI (Pixel)"]] * (float(slide.final_pixelsize**2))
        # print(df_out)
        # print("test_5")
        ratio = df_out["CI (Pixel)"].sum()/(df_out["Ventricle (Pixel)"].sum() + df_out["CI (Pixel)"].sum()) 
        # print("test_6")
        
        ci_count = np.count_nonzero(df_out["CI (Pixel)"])
        ve_count = np.count_nonzero(df_out["Ventricle (Pixel)"])

        ci_area  = df_out["CI (µm^2)"].sum()
        ve_area  = df_out["Ventricle (µm^2)"].sum()


        ci_max1  = df_out["CI (µm^2)"].sort_values( ascending= False).iloc[0]
        if ci_max1 != 0:
            ve_max1 = df_out[df_out["CI (µm^2)"] == ci_max1]["Ventricle (µm^2)"].iloc[0]
        else:
            ve_max1 = df_out["Ventricle (µm^2)"].max()

        ci_max2  = df_out["CI (µm^2)"].sort_values( ascending= False).iloc[1]

        if ci_max2 != 0:
            ve_max2 = df_out[df_out["CI (µm^2)"] == ci_max2]["Ventricle (µm^2)"].iloc[0]
        else:
            ve_max2 = df_out["Ventricle (µm^2)"].sort_values( ascending= False).iloc[1]


        df_slide.at[i,'Tot sections'] = ci_count + ve_count 
        df_slide.at[i,'Tot sections with CI'] = ci_count
        df_slide.at[i,'Tot sections with Ventricle'] = ve_count
        df_slide.at[i,'Tot ventricle area'] = ve_area
        df_slide.at[i,'Tot CI area'] = ci_area
        df_slide.at[i,'Tot ventricle + CI area'] = ve_area + ci_area                                                                                          
        df_slide.at[i,'Tot ratio pixel'] = ratio
        df_slide.at[i,'Ventricle size of section with Biggest IA 1'] = ve_max1
        df_slide.at[i,'Ventricle size of section with Biggest IA 2'] = ve_max2
        df_slide.at[i,'Biggest IA 1'] = ci_max1
        df_slide.at[i,'Biggest IA 2'] = ci_max2

        df_slide.to_excel(working_dir+"/Results/Q_LT_AFOG.xlsx")
        df_out.to_excel(working_dir+"/Results/Results_{}.xlsx".format(i.split("/")[-1].replace(".ndpi","")))

    except:
        print("!!!Failed Slide!!!", i)
        

D:/Nick/AFOGdata_P11703_MariaGalardi/20230928__01.czi
[(0, (275, 275, 512.0)), (1, (231, 231, 512.0)), (2, (185, 185, 512.0)), (3, (275, 275, 512.0)), (4, (275, 275, 512.0)), (5, (320, 320, 512.0)), (6, (185, 185, 512.0)), (7, (231, 231, 512.0)), (8, (275, 275, 512.0)), (9, (275, 275, 512.0)), (10, (275, 275, 512.0)), (11, (320, 320, 512.0)), (12, (275, 275, 512.0)), (13, (275, 275, 512.0)), (14, (230, 230, 512.0)), (15, (275, 275, 512.0)), (16, (321, 321, 512.0)), (17, (185, 185, 512.0)), (18, (185, 185, 512.0)), (19, (275, 275, 512.0)), (20, (186, 186, 512.0)), (21, (275, 275, 512.0)), (22, (320, 320, 512.0)), (23, (320, 320, 512.0)), (24, (185, 185, 512.0)), (25, (185, 185, 512.0))]
1/1 [==============================] - 6s 6s/step


C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__01.czi_17.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)
C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__01.czi_24.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)


    Section Number  Ventricle (Pixel)  CI (Pixel)
0                0               3020           0
1                1              21697           0
2                2              17353           0
3                3              27790           0
4                4              25288          15
5                5              31581        1830
6                6              19375           0
7                7              20482           0
8                8               2285           0
9                9              22017          90
10              10              26662           0
11              11              30618        1581
12              12              13210           0
13              13              25735        1290
14              14              23785          80
15              15              25119        4943
16              16              28027           0
17              17                211           0
18              18              11970           0


C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__06.czi_15.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)
C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__06.czi_21.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)


D:/Nick/AFOGdata_P11703_MariaGalardi/20230928__07.czi
[(0, (230, 230, 512.0)), (1, (140, 140, 512.0)), (2, (230, 230, 512.0)), (3, (185, 185, 512.0)), (4, (140, 140, 512.0)), (5, (185, 185, 512.0)), (6, (230, 230, 512.0)), (7, (185, 185, 512.0)), (8, (185, 185, 512.0)), (9, (185, 185, 512.0)), (10, (140, 140, 512.0)), (11, (140, 140, 512.0)), (12, (140, 140, 512.0)), (13, (185, 185, 512.0)), (14, (230, 230, 512.0)), (15, (230, 230, 512.0)), (16, (230, 230, 512.0)), (17, (185, 185, 512.0)), (18, (140, 140, 512.0)), (19, (140, 140, 512.0)), (20, (185, 185, 512.0)), (21, (185, 185, 512.0)), (22, (185, 185, 512.0)), (23, (185, 185, 512.0)), (24, (185, 185, 512.0)), (25, (185, 185, 512.0))]
1/1 [==============================] - 0s 60ms/step


...

    Section Number  Ventricle (Pixel)  CI (Pixel)
0                0              35124        4333
1                1              49018         555
2                2              22642          26
3                3               8252           0
4                4              42831           0
5                5               7152           0
6                6              35661        4402
7                7              40747           1
8                8              49610          70
9                9               9146           0
10              10              24996          40
11              11              10565           0
12              12              18439           0
13              13              48820           0
14              14              41774        5086
15              15              32991        1003
16              16               3477           0
17              17              19053           0
18              18              16523           0


C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__12.czi_3.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)
C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__12.czi_9.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)
C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__12.czi_15.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)
C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928

    Section Number  Ventricle (Pixel)  CI (Pixel)
0                0              26829        1022
1                1              38546        4991
2                2              14749           0
3                3                 14           0
4                4              33746        4230
5                5              40388        5505
6                6              23435        1285
7                7              10893           0
8                8              33400        4033
9                9                330           0
10              10              17661           0
11              11               9065           0
12              12              37306        6668
13              13              30726        1586
14              14              23414           2
15              15                172           0
16              16              20789           0
17              17              30569        1905
18              18              39185        6571


C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__13.czi_15.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)
C:\Users\pa20c964\AppData\Local\Temp\ipykernel_33580\1221692010.py:33: UserWarning: D:\Nick\AFOGdata_P11703_MariaGalardi/Annotations/20230928__13.czi_21.tif is a low contrast image
  imsave(working_dir+"/Annotations/{}_{}.tif".format(i.split("/")[-1].replace(".ndpi",""),pred_n),pred_i)


    Section Number  Ventricle (Pixel)  CI (Pixel)
0                0              30786         144
1                1               5717         168
2                2              19501           0
3                3              27792           0
4                4              31550           0
5                5              26670           0
6                6              10988           0
7                7              30122          10
8                8               7213           0
9                9              28992           0
10              10              25952           0
11              11               9804           0
12              12              20189           0
13              13              27168           0
14              14              13335           0
15              15                 24           0
16              16              33260           0
17              17              27273         504
18              18              23018           0


In [9]:
print("annotation completed")

annotation completed
